# 4 Bootstrap Distributions

Learn how to use resampling to perform bootstrapping, used to estimate variation in an unknown population. Understand the difference between sampling distributions and bootstrap distributions.

- Using the spotify_population dataset, calculate the mean duration in minutes. Call the calculated column mean_dur.
- Using the spotify_sample dataset, perform the same calculation in another column called mean_dur.
- Look at the two values. How different are they?

# Principles of bootstrapping

Bootstrapping is, in some sense, the opposite of sampling from a population. Sampling treats your dataset as the population, and you generate a random subset. Bootstrapping treats your dataset as a sample and uses it to build up a theoretical population.

# With or without replacement

So far in the course, you've seen sampling with and without replacement. It's important to know when to use each method.

# Generating a bootstrap distribution

The process for generating a bootstrap distribution is remarkably similar to the process for generating a sampling distribution; only the first step is different.

To make a sampling distribution, you start with the population and sample without replacement. To make a bootstrap distribution, you start with a sample and sample that with replacement. After that, the steps are the same: calculate the summary statistic that you are interested in on that sample/resample, then replicate the process many times. In each case, you can visualize the distribution with a histogram.

Here, spotify_sample is a subset of the spotify_population dataset. To make it easier to see how resampling works, a row ID column has been added, and only the artist name, song name, and danceability columns have been included.

spotify_sample is available; dplyr and ggplot2 are loaded.

# Instructions:

- Generate a single bootstrap resample from spotify_sample.

In [ ]:
# Generate 1 bootstrap resample
spotify_1_resample <- spotify_sample %>%  slice_sample(prop = 1, replace = TRUE)


# See the result
spotify_1_resample

- Summarize to calculate the mean danceability of spotify_1_resample as mean_danceability, then pull out this value to get a numeric vector of length 1.

In [ ]:
# From previous step
spotify_1_resample <- spotify_sample %>% 
  slice_sample(prop = 1, replace = TRUE)

# Calculate mean danceability of resample
mean_danceability_1 <- spotify_1_resample %>% 
  summarize(mean_danceability = mean(danceability)) %>% 
  pull(mean_danceability)



# See the result
mean_danceability_1

- Replicate the expression provided 1000 times.

In [ ]:
# Replicate this 1000 times
mean_danceability_1000 <- replicate(
  n=1000,
  expr = {
    spotify_1_resample <- spotify_sample %>% 
      slice_sample(prop = 1, replace = TRUE)
    spotify_1_resample %>% 
      summarize(mean_danceability = mean(danceability)) %>% 
      pull(mean_danceability)})

# See the result
mean_danceability_1000

- Store mean_danceability_1000 in a tibble, in a column named resample_mean.
- Using bootstrap_distn, draw a histogram of the resample_means with binwidth 0.002.

In [ ]:
# From previous steps
mean_danceability_1000 <- load_step_4()

# Store the resamples in a tibble
bootstrap_distn <- tibble(
  resample_mean = mean_danceability_1000)

# Draw a histogram of the resample means with binwidth 0.002
ggplot(bootstrap_distn, aes(resample_mean)) + geom_histogram(binwidth = 0.002)

# Bootstrap statistics and population statistics

Bootstrap distribution statistics can be used to estimate population parameters. But can you always rely on them to give an accurate estimate of an unknown population parameter?

Should the mean and the standard deviation of the bootstrap distribution both be used to estimate the corresponding values of the population?

# Possible Answers

(x) No, the mean of the bootstrap distribution will always be near the sample mean, which may not necessarily be very close to the population mean.

( ) No, the standard deviation of the bootstrap distribution (if divided by the square root of the sample size) will tend to be near the sample standard deviation, which may not necessarily be very close to the population standard deviation.

( ) Yes, both estimates should match up closely with the population values in all scenarios.

( ) Yes, the variability of the sample, the population, the bootstrap distribution, and the sampling distribution will all be similar regardless of sample size selected.

# Sampling distribution vs. bootstrap distribution

The sampling distribution and bootstrap distribution are closely linked. In situations where you can repeatedly sample from a population (these occasions are rare) and as you learn about both, it's helpful to generate both the sampling distribution and the bootstrap distribution, one after the other, to see how they are related.

Here, the statistic you are interested in is the mean popularity score of the songs.

spotify_population (the whole dataset) and spotify_sample (500 rows only representing an original sample) are available; dplyr is loaded.

# Instructions:

- Generate a sampling distribution of 2000 replicates.
- Sample 500 rows of the population without replacement.
- Calculate the statistic of interest (the mean popularity) in the column mean_popularity.
- Pull out the statistic so it is a single numeric value (not a tibble).

In [ ]:
# Generate a sampling distribution
mean_popularity_2000_samp <- replicate(
  # Use 2000 replicates
  n=2000,
  expr = {
    # Start with the population
    spotify_population %>% 
      # Sample 500 rows without replacement
      slice_sample(n = 500) %>% 
      # Calculate the mean popularity as mean_popularity
      summarize(mean_popularity = mean(popularity)) %>% 
      # Pull out the mean popularity
      pull(mean_popularity)})
# See the result
head(mean_popularity_2000_samp)

- Generate a bootstrap distribution of 2000 replicates.
- Sample 500 rows of the sample with replacement.
- Calculate the statistic of interest (the mean popularity) in the column mean_popularity.
- Pull out the statistic so it is a single numeric value (not a tibble).

In [ ]:
# Generate a bootstrap distribution
mean_popularity_2000_boot <- replicate(
  # Use 2000 replicates
  n = 2000,
  expr = {
    # Start with the sample
    spotify_sample %>% 
      # Sample same number of rows with replacement
      slice_sample(prop = 1, replace = TRUE) %>% 
      # Calculate the mean popularity
      summarize(mean_popularity = mean(popularity)) %>% 
      # Pull out the mean popularity
      pull(mean_popularity)
  }
)

# See the result
mean_popularity_2000_boot

# Compare sampling and bootstrap means

To make calculation easier, the distributions from the previous exercise have been included in tibbles. mean_popularity_2000_samp is in the sample_mean column of sampling_distribution, and mean_popularity_2000_boot is in the resample_mean column of bootstrap_distribution.

spotify_population, spotify_sample, sampling_distribution, and bootstrap_distribution are available; dplyr is loaded.

# Instructions:

Calculate the mean popularity with summarize() in 4 ways.

- Population: from spotify_population, take the mean of popularity.
- Sample: from spotify_sample, take the mean of popularity.
- Sampling distribution: from sampling_distribution, take the mean of sample_mean.
- Bootstrap distribution: from bootstrap_distribution, take the mean of resample_mean.

In [ ]:
# Calculate the true population mean popularity
pop_mean <- spotify_population %>% 
  summarize(mean(popularity))

# Calculate the original sample mean popularity
samp_mean <- spotify_sample %>% 
  summarize(mean(popularity))

# Calculate the sampling dist'n estimate of mean popularity
samp_distn_mean <- sampling_distribution %>% 
  summarize(mean(sample_mean))

# Calculate the bootstrap dist'n estimate of mean popularity
boot_distn_mean <- bootstrap_distribution %>% 
  summarize(mean(resample_mean))

# See the results
c(pop = pop_mean, samp = samp_mean, samp_distn = samp_distn_mean, boot_distn = boot_distn_mean)

# Question

Based on the four means you just calculated, which statement is true?

# Possible answers

( ) The sampling distribution mean is closest to the original sample mean; the bootstrap distribution mean is the best estimate of the true population mean.

(x) The sampling distribution mean is the best estimate of the true population mean; the bootstrap distribution mean is closest to the original sample mean.

( ) The sampling distribution mean and the bootstrap distribution mean are equally good estimates of the original sample mean.

( ) The sampling distribution mean and the bootstrap distribution mean are equally good estimates of the true population mean.

In [ ]:
DM.result = 2

# Compare sampling and bootstrap standard deviations

In the same way that you looked at how the sampling distribution and bootstrap distribution could be used to estimate the population mean, you'll now take a look at how they can be used to estimate variation, or more specifically, the standard deviation, in the population.

spotify_population, spotify_sample, sampling_distribution, and bootstrap_distribution are available; dplyr is loaded.

# Instructions:

Calculate the standard deviation of popularity with summarize() in 4 ways.

- Population: from spotify_population, take the standard deviation of popularity.
- Original sample: from spotify_sample, take the standard deviation of popularity.
- Sampling distribution: from sampling_distribution, take the standard deviation of sample_mean and multiply by the square root of the sample size (500).
- Bootstrap distribution: from bootstrap_distribution, take the standard deviation of resample_mean and multiply by the square root of the sample size.

In [ ]:
# Calculate the true population std dev popularity
pop_sd <- spotify_population %>% 
  summarize(sd(popularity))

# Calculate the original sample std dev popularity
samp_sd <- spotify_sample %>% 
  summarize(sd(popularity))

# Calculate the sampling dist'n estimate of std dev popularity
samp_distn_sd <- sampling_distribution %>% 
  summarize(sd(sample_mean) * sqrt(500))

# Calculate the bootstrap dist'n estimate of std dev popularity
boot_distn_sd <- bootstrap_distribution %>% 
  summarize(sd(resample_mean) * sqrt(500))

# See the results
c(pop = pop_sd, samp = samp_sd, sam_distn = samp_distn_sd, boot_distn = boot_distn_sd)

# Question

Based on the four results you just calculated, which statement is true?

# Possible answers

( ) The calculation from the sampling distribution is closest to the original sample standard deviation; the calculation from the bootstrap distribution is the best estimate of the true population standard deviation.

( ) The sample standard deviation is the best estimate of the true population standard deviation; the calculation from the bootstrap distribution is closest to the original sample standard deviation.

( ) The calculations from both the sampling distribution and from the bootstrap distribution are equally close to the original sample standard deviation.

(x) While not the closest of the values given, the calculation from the bootstrap distribution provides a good estimate of the true population standard deviation.

In [ ]:
DM.result = 4

# Confidence interval interpretation

When reporting results, it is common to provide a confidence interval alongside an estimate.

What does that confidence interval provide?

# Possible Answers

(x) A range of all values that a variable/column from a sample may take on.

( ) All numbers between 0 and 1.

( ) A range of plausible values for a variable measured in our population (such as popularity in spotify_population).

( ) A range of plausible values for an unknown quantity.

# Calculating confidence intervals

We can use the cumulative distribution function and its inverse to calculate confidence intervals in R. You'll do so with the Spotify data now.

# Instructions:

- Generate a 95% confidence interval using the quantile method.

- Summarize to get the 0.025 quantile as lower, and the 0.975 quantile as upper.

In [ ]:
# Generate a 95% confidence interval using the quantile method
conf_int_quantile <- bootstrap_distribution %>% 
  summarize(
    lower = quantile(resample_mean, 0.025),
    upper = quantile(resample_mean, 0.975)
  )

# See the result
conf_int_quantile

Generate a 95% confidence interval using the standard error method.

- Calculate point_estimate as the mean of resample_mean, and standard_error as the standard deviation of resample_mean.
- Calculate lower as the 0.025 quantile of an inv. CDF from a normal distribution with mean point_estimate and standard deviation standard_error.
- Calculate upper as the 0.975 quantile of that same inv. CDF.

In [ ]:
# Generate a 95% confidence interval using the std error method
conf_int_std_error <- bootstrap_distribution %>% 
  summarize(
    point_estimate = mean(resample_mean),
    standard_error = sd(resample_mean),
    lower = qnorm(0.025, point_estimate, standard_error),
    upper = qnorm(0.975, point_estimate, standard_error)
  )

# See the result
conf_int_std_error